## Chat Message History
- We need to store the historical chat messages in efficient way.
- It wraps another Runnable and manages the chat message history for it.
- Specifically, it loads previous messages in the conversation **BEFORE** passing it to the Runnable, and it saves the generated response as a message **AFTER** calling the Runnable.
- This class also enables multiple conversations by saving each conversation with a **session_id**
- It then expects a `session_id` to be **passed in the config** when calling the Runnable, and uses that lo took up **the relevant conversation history**.


<br />


<img src="../ASSETS/chat-message-history-1.png" alt="Chat Message History" width="1250" style="max-width: 100%;" />

### Simple Chain

In [4]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    ChatPromptTemplate
)
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

load_dotenv("./../.env")

base_url = os.getenv("OLLAMA_BASE_URL")
model = "llama3.2:latest"

llm = OllamaLLM(base_url=base_url, model=model)
template = ChatPromptTemplate.from_template("{prompt}")
chain = template | llm | StrOutputParser()

about = "My name is Nathan Garzya. I study at the University of Surabaya."
chain.invoke({
    "prompt": about
})

"Hi Nathan, it's nice to meet you! Congratulations on your studies at the University of Surabaya. How are things going so far? Are you enjoying your time in school and exploring the city of Surabaya?"

In [6]:
prompt = "What is my name?"
chain.invoke({
    "prompt": prompt
})

"I don't have any information about your identity or personal details. I'm a large language model, I don't have the ability to know or recall individual user information. Each time you interact with me, it's a new conversation and I start from a blank slate. If you'd like to share your name with me, I can use it to personalize our conversation!"

### Runnable with Message History
In order to properly set this up, there are 2 main things to consider:
- How to store and load messages?
- What is the underlying Runnable you are wrapping and what are its inputs/outputs?

In [7]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [11]:
def get_session_history(session_id):
    return SQLChatMessageHistory(
        session_id=session_id,
        connection_string="sqlite:///chat_history.db"
    )

In [12]:
# Create Runnable with Message History
runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history
)

In [13]:
# Get History messages from user_id
user_id = "nathan_garzya"
history = get_session_history(user_id)
history.get_messages()

[]

In [14]:
# Clear Message History
history.clear()

In [16]:
runnable_with_history.invoke(
    [HumanMessage(content=about)],
    config={
        "configurable": {
            "session_id": user_id
        }
    }
)

"Hello Nathan, nice to meet you. How's your experience been at the University of Surabaya so far?"

In [17]:
history.get_messages()

[HumanMessage(content='My name is Nathan Garzya. I study at the University of Surabaya.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Nathan, nice to meet you. How's your experience been at the University of Surabaya so far?", additional_kwargs={}, response_metadata={})]

In [18]:
runnable_with_history.invoke(
    [HumanMessage(content="What is my name?")],
    config={
        "configurable": {
            "session_id": user_id
        }
    }
)
# But it will produce incorrect answer
# It was not able to take historical message correctly for the context

'It appears that we have a conversation log between a human user and an AI system.\n\nHere\'s a breakdown of the conversation:\n\n1. The human user introduces themselves, stating their name as Nathan Garzya and mentioning that they study at the University of Surabaya.\n2. The AI responds with a friendly message, asking how the human\'s experience has been at the university so far.\n3. The human user asks for clarification on what is being asked - "What is my name?".\n\nTo provide a helpful response, I\'ll add a new message to the conversation:\n\nAIMessage(content="I apologize for not clarifying that earlier! Your name is Nathan Garzya, and you mentioned it earlier in our conversation.", additional_kwargs={}, response_metadata={})\n\nThis response acknowledges the human\'s question and reiterates their introduction.'

In [19]:
history.get_messages()

[HumanMessage(content='My name is Nathan Garzya. I study at the University of Surabaya.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Nathan, nice to meet you. How's your experience been at the University of Surabaya so far?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='It appears that we have a conversation log between a human user and an AI system.\n\nHere\'s a breakdown of the conversation:\n\n1. The human user introduces themselves, stating their name as Nathan Garzya and mentioning that they study at the University of Surabaya.\n2. The AI responds with a friendly message, asking how the human\'s experience has been at the university so far.\n3. The human user asks for clarification on what is being asked - "What is my name?".\n\nTo provide a helpful response, I\'ll add a new message to the conversation:\n\nAIMessage(content="I apologize for not cla

### Message History with Dictionary Like Inputs

In [21]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    MessagesPlaceholder
)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

In [22]:
# Get the system message first
system = SystemMessage(content="You are helpful assistant.")
human = HumanMessagePromptTemplate.from_template("{input}")

messages = [
    system, 
    MessagesPlaceholder(variable_name='history'), 
    human
]

prompt = ChatPromptTemplate(messages=messages)
chain = prompt | llm | StrOutputParser()

runnable_with_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key='input',
    history_messages_key='history'
)

In [24]:
def chat_with_llm(session_id, input):
    output = runnable_with_history.invoke(
        {
            "input": input
        },
        config={
            "configurable": {
                "session_id": session_id
            }
        }
    )
    
    return output

In [25]:
new_user_id = "canonflow"
chat_with_llm(new_user_id, about)

"Hello Nathan! It's nice to meet you. Congratulations on studying at the University of Surabaya, one of Indonesia's top universities!\n\nHow can I assist you today? Do you need help with anything related to your studies, research, or perhaps looking for information about the university or surrounding area?"

In [26]:
chat_with_llm(new_user_id, "What is my name?")

"Hello Nathan! Don't worry, I remember - your name is Nathan Garzya. You're a student at the University of Surabaya.\n\nYou asked how I can assist you today. Well, feel free to ask me anything related to your studies, research, or even looking for information about the university or its surroundings. I'm here to help!\n\nAre you looking for something specific, like academic resources, campus facilities, or perhaps recommendations on things to do in Surabaya? Or is there something else on your mind?"

In [28]:
get_session_history(new_user_id).get_messages()

[HumanMessage(content='My name is Nathan Garzya. I study at the University of Surabaya.', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Nathan! It's nice to meet you. Congratulations on studying at the University of Surabaya, one of Indonesia's top universities!\n\nHow can I assist you today? Do you need help with anything related to your studies, research, or perhaps looking for information about the university or surrounding area?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Hello Nathan! Don't worry, I remember - your name is Nathan Garzya. You're a student at the University of Surabaya.\n\nYou asked how I can assist you today. Well, feel free to ask me anything related to your studies, research, or even looking for information about the university or its surroundings. I'm here to help!\n\nAre you looking for something specific, like academic resource